[Reference](https://blog.stackademic.com/best-practices-for-structuring-your-fastapi-projects-e66482b27d02)

# 1. Basic Structure of a FastAPI Project
```
my_project/
├── app/
│   ├── main.py
│   ├── api/
│   │   ├── __init__.py
│   │   └── controllers/
│   │       ├── __init__.py
│   │       └── items.py
│   ├── core/
│   │   ├── __init__.py
│   │   └── config.py
│   ├── models/
│   │   ├── __init__.py
│   │   └── item.py
│   ├── schemas/
│   │   ├── __init__.py
│   │   └── item.py
│   ├── crud/
│   │   ├── __init__.py
│   │   └── item.py
│   ├── db/
│   │   ├── __init__.py
│   │   └── session.py
│   └── tests/
│       ├── __init__.py
│       └── test_items.py
├── .env
├── requirements.txt
└── README.md
```

# 2. Details of Folder Structure

## 2.1 main.py

In [1]:
# main.py
from fastapi import FastAPI
from app.api.controllers import items

app = FastAPI()

app.include_router(items.router)

@app.get("/")
def read_root():
    return {"message": "Welcome to FastAPI"}

## 2.2 api/

In [2]:
# items.py in api/controllers
from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
from app.schemas.item import Item
from app.crud.item import get_item, create_item
from app.db.session import get_db

router = APIRouter()

@router.post("/items/", response_model=Item)
def create_new_item(item: Item, db: Session = Depends(get_db)):
    db_item = get_item(db, item_id=item.id)
    if db_item:
        raise HTTPException(status_code=400, detail="Item already exists")
    return create_item(db=db, item=item)

## 2.3 core/

In [3]:
# config.py in core/
import os
from dotenv import load_dotenv

load_dotenv()

class Settings:
    PROJECT_NAME: str = "FastAPI Project"
    SQLALCHEMY_DATABASE_URI: str = os.getenv("DATABASE_URL")

settings = Settings()

## 2.4 models/

In [4]:
# item.py in models/
from sqlalchemy import Column, Integer, String
from app.db.session import Base

class Item(Base):
    __tablename__ = "items"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    description = Column(String, index=True)

## 2.5 schemas/

In [5]:
# item.py in schemas/
from pydantic import BaseModel

class Item(BaseModel):
    id: int
    name: str
    description: str

    class Config:
        orm_mode = True

## 2.6 crud/

In [6]:
# item.py in crud/
from sqlalchemy.orm import Session
from app.models.item import Item as ItemModel
from app.schemas.item import Item as ItemSchema

def get_item(db: Session, item_id: int):
    return db.query(ItemModel).filter(ItemModel.id == item_id).first()

def create_item(db: Session, item: ItemSchema):
    db_item = ItemModel(id=item.id, name=item.name, description=item.description)
    db.add(db_item)
    db.commit()
    db.refresh(db_item)
    return db_item

## 2.7 db/

In [7]:
# session.py in db/
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from app.core.config import settings

SQLALCHEMY_DATABASE_URL = settings.SQLALCHEMY_DATABASE_URI

engine = create_engine(SQLALCHEMY_DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

## 2.8 tests/

In [8]:
# test_items.py in tests/
from fastapi.testclient import TestClient
from app.main import app

client = TestClient(app)

def test_create_item():
    response = client.post("/items/", json={"id": 1, "name": "Item 1", "description": "A sample item"})
    assert response.status_code == 200
    assert response.json()["name"] == "Item 1"